In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
#use some library
import pandas as pd
import numpy as np

#read Excel-file and change some data, for correcting founding
df = pd.read_excel('data/Dannye_Po_Naseleniyu.xlsx')

df['name'] = df.loc[:, 'name'].replace(regex=True,to_replace="г. ", value="")
df['name'] = df.loc[:, 'name'].replace(regex=True,to_replace="г.п. ", value="")

#add population coefficients 
max_value = df[2019].max() + 1

df['trend_popul'] = np.round(df[2019] / df[2015],1)
df['cof_popul'] = (df[2019] // (max_value / 10) / 10) + 0.1

# add salary coefficients
average2019 = 2.1598
average2016 = 18.569
avgincountry = 981.6 / average2019
print(avgincountry)

df['salary_2019'] = np.round(df['salary_2019'] / average2019,2)
df['salary_2016'] = np.round(df['salary_2016'] / average2016,2) 
df['trend_salary'] = np.round(df['salary_2019'] / df['salary_2016'],1)
df['cof_salary'] = np.round(df['salary_2019'] / avgincountry,1)
df.head()

454.48652653023424


,name,2019,2015,region,salary_2019,salary_2016,trend_popul,cof_popul,trend_salary,cof_salary
0,Береза,29437,29500,Брестская область,428.93,307.76,1.0,0.3,1.4,0.9
1,Ганцевичи,13949,14043,Брестская область,358.37,249.04,1.0,0.2,1.4,0.8
2,Дрогичин,14931,14800,Брестская область,366.98,249.84,1.0,0.2,1.5,0.8
3,Жабинка,14577,13353,Брестская область,400.18,294.07,1.1,0.2,1.4,0.9
4,Иваново,16435,16597,Брестская область,382.21,262.01,1.0,0.2,1.5,0.8


In [4]:
# use geopy and add coordinates for all cities
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="ny_explorer", timeout = 3)
latitude_column = []
longitude_column = []

def get_location(df_change):
    for index, row in df_change.iterrows():
        location = geolocator.geocode(row['name'] + ", " + row['region'])
        if location != None:            
            latitude_column.append(location.latitude)
            longitude_column.append(location.longitude)
        else:
            latitude_column.append('None')
            longitude_column.append('None')
            print(row['name'] + ", " + row['region'] + '  not found')

get_location(df) 

df['latitude'] = latitude_column
df['longitude'] = longitude_column

df = df.loc[df['latitude'] != 'None']

df.shape

Краснополье, Могилёвская область  not found


(102, 12)

In [5]:
#create map of Belarus with all cities-point
import folium

address = 'Belarus'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Belarus are {}, {}.'.format(latitude, longitude))

# create map of Belarus using latitude and longitude values
map_belarus = folium.Map(location=[latitude, longitude], zoom_start=6.1)

# add markers to map
for lat, lng, city in zip(df['latitude'], df['longitude'], df['name']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_belarus)  
    
map_belarus

The geograpical coordinate of Belarus are 53.4250605, 27.6971358.


In [6]:
#connect Foursquare and test on one city
import requests

CLIENT_ID = 'RAPB5Z0OPLAX11CL3QCXA1MV5RQWH5GGCC4UPZMAJGD3WSNA' # your Foursquare ID
CLIENT_SECRET = 'QXQRL1HLS3OLX2UUG5M33WNOPTQ2BA1PECREBK1MQLU34TNK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

df.loc[0, 'name']

city_latitude = df.loc[0, 'latitude'] # neighborhood latitude value
city_longitude = df.loc[0, 'longitude'] # neighborhood longitude value

city_name = df.loc[0, 'name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']

from pandas.io.json import json_normalize   

nearby_venues = json_normalize(venues) # flatten JSON

Your credentails:
CLIENT_ID: RAPB5Z0OPLAX11CL3QCXA1MV5RQWH5GGCC4UPZMAJGD3WSNA
CLIENT_SECRET:QXQRL1HLS3OLX2UUG5M33WNOPTQ2BA1PECREBK1MQLU34TNK
Latitude and longitude values of Береза  are 52.5345683, 24.9844968.


In [ ]:
#use Foursquare for add all venues in this cities
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'city Latitude', 
                  'city Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Belarus_cities_venues = getNearbyVenues(names=df['name'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

print(Belarus_cities_venues.shape)
Belarus_cities_venues.sample()

In [ ]:
# newtable = Belarus_cities_venues.groupby(["Venue Category"]).agg('count').sort_values('name', ascending=False)
# newtable.head(10)
# newtable.sample(10)

In [ ]:
Belarus_cities_venues.groupby('name').count()

Belarus_cities_onehot = pd.get_dummies(Belarus_cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
Belarus_cities_onehot['name'] = Belarus_cities_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [Belarus_cities_onehot.columns[-1]] + list(Belarus_cities_onehot.columns[:-1])

Belarus_cities_onehot = Belarus_cities_onehot[fixed_columns]

Belarus_cities_grouped = Belarus_cities_onehot.groupby('name').max().reset_index()

Belarus_cities_grouped.head()

# check data by one criteria
# Belarus_cities_grouped_df = Belarus_cities_grouped.loc[Belarus_cities_grouped['name'] == 'Житковичи '] 
# Belarus_cities_grouped_df.head()

In [ ]:
Belarus_cities_union = df

Belarus_cities_union = Belarus_cities_union.join(Belarus_cities_grouped.set_index('name'), on='name')
Belarus_cities_union = Belarus_cities_union.fillna(0)

Belarus_cities_union.head()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
kclusters = 5

Belarus_cities_venues_sorted = Belarus_cities_union[{'name','cof_popul','cof_salary','Restaurant','trend_popul','trend_salary'}]

Belarus_cities_grouped_clustering = Belarus_cities_venues_sorted.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(Belarus_cities_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

# add clustering labels
Belarus_cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Belarus_cities_venues_sorted = Belarus_cities_venues_sorted.drop(['cof_popul','cof_salary','trend_popul','trend_salary'],axis = 1)

# merge df with Belarus_cities_venues_sorted
Belarus_cities_merged = df
Belarus_cities_merged = Belarus_cities_merged.join(Belarus_cities_venues_sorted.set_index('name'), on='name')

Belarus_cities_merged = Belarus_cities_merged.dropna()

Belarus_cities_merged['Cluster Labels'] = Belarus_cities_merged['Cluster Labels'].astype('int')
Belarus_cities_merged['Cluster Labels'].unique()

Belarus_cities_merged = Belarus_cities_merged.drop([2015,'region','salary_2016'],axis = 1)

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Belarus_cities_merged['latitude'], Belarus_cities_merged['longitude'], Belarus_cities_merged['name'], Belarus_cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
#show information for all clusters
Belarus_cities_grouped_df = Belarus_cities_merged.loc[Belarus_cities_merged['Cluster Labels'] == 0] 
Belarus_cities_grouped_df.head(10)

In [ ]:
Belarus_cities_grouped_df1 = Belarus_cities_merged.loc[Belarus_cities_merged['Cluster Labels'] == 1] 
Belarus_cities_grouped_df1.head(10)

In [ ]:
Belarus_cities_grouped_df2 = Belarus_cities_merged.loc[Belarus_cities_merged['Cluster Labels'] == 2] 
Belarus_cities_grouped_df2.head(10)

In [ ]:
Belarus_cities_grouped_df3 = Belarus_cities_merged.loc[Belarus_cities_merged['Cluster Labels'] == 3] 
Belarus_cities_grouped_df3.head(10)

In [ ]:
Belarus_cities_grouped_df3 = Belarus_cities_merged.loc[Belarus_cities_merged['Cluster Labels'] == 4] 
Belarus_cities_grouped_df3.head(10)